In [1]:
%%time
from google.colab import drive
drive.mount('/content/drive')
%cd "drive/My Drive/pytorch/評価"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/pytorch/評価
CPU times: user 325 ms, sys: 129 ms, total: 454 ms
Wall time: 25.2 s


In [2]:
%%time
! npm install -g localtunnel
!python3 -m pip install visdom
!python3 -m visdom.server -port 8076 >> visdomlog.txt 2>&1 &
!lt --port 8076 >> url.txt 2>&1 &

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.0
added 35 packages from 21 contributors in 1.779s
     |████████████████████████████████| 686kB 3.4MB/s 
     |████████████████████████████████| 204kB 56.6MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655252 sha256=af707a2802fb2e2ed7931e409593af515e558b2fb3148538ea1aeb11f4bcc550
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=b636fb67d2d7238755fdb90295e5728cc7e4f0964a798742ef700ab85aa8d66c
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile
CPU times: user 91.8 ms, sys: 31.2 ms, total: 123 ms
Wall time: 16.7 s


In [3]:
%%time
import time
time.sleep(5)
! cat url.txt

CPU times: user 10.9 ms, sys: 5.07 ms, total: 16 ms
Wall time: 6.12 s


In [0]:
from lib import network_finetuning as network
from lib import architecture
from lib import dataloader as dl
from lib import optimizer
from lib import trainer

import torchvision

In [5]:
%%time
net = network.ResNet50(class_size=10)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
%%time
batch_size = 128
#transform = dl.pattern_transform(resize=224, HorizontalFlip=True, VerticalFlip=False, Rotation=True, Perspective=True, Crop=True, Erasing=True)
transform = dl.pattern_transform(resize=224, HorizontalFlip=True, VerticalFlip=True, Rotation=True, Perspective=True, Crop=False, Erasing=True)
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=dl.val_transform())
data_loader = dl.DataLoader(train_dataset, test_dataset, batch_size=batch_size,suffle=True, num_workers=2)

[Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR), RandomHorizontalFlip(p=0.5), RandomVerticalFlip(p=0.5), RandomRotation(degrees=(-180, 180), resample=False, expand=False), RandomPerspective(p=0.5), ToTensor(), <torchvision.transforms.transforms.RandomErasing object at 0x7fb2e83f6630>, Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))]
Files already downloaded and verified
Files already downloaded and verified
CPU times: user 1.4 s, sys: 571 ms, total: 1.97 s
Wall time: 8.94 s


In [7]:
%%time
model = architecture.CNN_Architecture(net)

CPU times: user 2.11 ms, sys: 15.2 ms, total: 17.3 ms
Wall time: 47.9 ms


In [8]:
%%time
model.train(data_loader, epoch_count=50)

cuda


Setting up a new session...


----------
Epoch 1/50 2020-01-29 12:31:24.653404
val Loss: 5.1098 Acc: 0.1000
----------
Epoch 2/50 2020-01-29 12:31:43.870955
train Loss: 2.0248 Acc: 0.2516
val Loss: 1.9921 Acc: 0.2732
----------
Epoch 3/50 2020-01-29 12:36:18.106200
train Loss: 1.8438 Acc: 0.3206
val Loss: 1.7723 Acc: 0.3648
----------
Epoch 4/50 2020-01-29 12:40:52.990840
train Loss: 1.7271 Acc: 0.3710
val Loss: 1.6564 Acc: 0.3946
----------
Epoch 5/50 2020-01-29 12:45:28.096293
train Loss: 1.6452 Acc: 0.4020
val Loss: 1.5757 Acc: 0.4312
----------
Epoch 6/50 2020-01-29 12:50:03.757434
train Loss: 1.5693 Acc: 0.4343
val Loss: 1.5210 Acc: 0.4639
----------
Epoch 7/50 2020-01-29 12:54:38.342656
train Loss: 1.4846 Acc: 0.4621
val Loss: 1.4077 Acc: 0.5049
----------
Epoch 8/50 2020-01-29 12:59:13.377359
train Loss: 1.4119 Acc: 0.4942
val Loss: 1.3316 Acc: 0.5267
----------
Epoch 9/50 2020-01-29 13:03:48.387823
train Loss: 1.3413 Acc: 0.5212
val Loss: 1.4399 Acc: 0.5132
----------
Epoch 10/50 2020-01-29 13:08:23.237626


In [1]:
%%time
model.predict(data_loader['val'])

NameError: ignored